In [ ]:
from copy import copy 
from matplotlib import pyplot as plt
import datetime

import numpy as np
import pandas as pd

import summer2
from summer.utils import ref_times_to_dti

from autumn.core.project import get_project, load_timeseries
from autumn.core.project.params import get_with_nested_key

# Targets represent data we are trying to fit to
from estival import targets as est
# We specify parameters using (Bayesian) priors
from estival import priors as esp
# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

# Some preliminary code

### Load project, model and targets

In [ ]:
project = get_project("sm_covid2", "france")
death_target = project.calibration.targets[0]

default_configuration = project.param_set.baseline
m = project.build_model(default_configuration.to_dict()) 

In [ ]:
targets = [
    est.NegativeBinomialTarget("infection_deaths", death_target.data, dispersion_param=100.0)
]

### Format priors to use in estival

In [ ]:
def get_estival_uniform_priors(autumn_priors):
    estival_priors = []
    for prior_dict in autumn_priors:
        assert prior_dict["distribution"] == "uniform", "Only uniform priors are currently supported"
        
        if not "random_process.delta_values" in prior_dict["param_name"] and not "dispersion_param" in prior_dict["param_name"]:
            estival_priors.append(
                esp.UniformPrior(prior_dict["param_name"], prior_dict["distri_params"]),
            )
            
    ndelta_values = len([prior_dict for prior_dict in autumn_priors if prior_dict["param_name"].startswith("random_process.delta_values")])
    
    estival_priors.append(esp.UniformPrior("random_process.delta_values", [-2.0,2.0], ndelta_values + 1))
    
    return estival_priors
    

In [ ]:
priors = get_estival_uniform_priors(project.calibration.all_priors)

In [ ]:
default_params = m.builder.get_default_parameters()

### Create BayesianCompartmentalModel object

In [ ]:
bcm = BayesianCompartmentalModel(m, default_params, priors, targets)

### Helper function to plot model fit to data

In [ ]:
def plot_fit(bcm, params):

    REF_DATE = datetime.date(2019,12,31)
    datetime_target = copy(bcm.targets["infection_deaths"].data)
    datetime_target.index = ref_times_to_dti(REF_DATE, datetime_target.index)

    ax = bcm.run(params).derived_outputs["infection_deaths"].plot()
    datetime_target.plot(style='.')
    ll = bcm.loglikelihood(**params)

    text = f"ll={ll}"
    plt.text(0.8, 0.9, text, transform=ax.transAxes)

# Compare model fits

In [ ]:
import numpy as np

In [ ]:
example_params = {
    'contact_rate': 0.04104869232093793,
    'infectious_seed_time': 30.447925441292778,
    'age_stratification.ifr.multiplier': 0.8145117432172231,
    'voc_emergence.delta.new_voc_seed.time_from_gisaid_report': 18.52679245171894,
    'voc_emergence.omicron.new_voc_seed.time_from_gisaid_report': 0.0,
    'random_process.noise_sd': 0.5835215881701278,
    "random_process.delta_values": np.array([
0.0,
-1.306496185258189,
 0.11250765346332026,
 0.5009919549420943,
 -0.06375110616249913,
 -0.24028730546991617,
0.2606697551298094,
-0.13194034829544132,
-0.3559750720412327,
0.04754597441674813,
 0.12138039221640762,
0.6411075116621299,  
        0.0,0.0,0.0,0.0,0.0,0.0,
    ]),
}

In [ ]:
plot_fit(bcm, example_params)

In [ ]:
test_rp = example_params["random_process.delta_values"].copy()
test_rp[1] = -2.3
curp = example_params | {"random_process.delta_values": test_rp}

In [ ]:
plot_fit(bcm, curp)

In [ ]:
from estival.optimization import nevergrad as eng

In [ ]:
import nevergrad as ng

In [ ]:
opt = eng.optimize_model(bcm, obj_function=bcm.loglikelihood)

In [ ]:
# It is usually a good to idea to 'prime' the optimizer with a moderate number of runs,
# since nevergrad seems use this first budget in its decision making about optimization hyper-parameters
# (...as far as I can tell)
res = opt.minimize(2000)
print(res.value[1])
plot_fit(bcm, res.value[1])

In [ ]:
res = opt.minimize(6000)
print(res.value[1])
plot_fit(bcm, res.value[1])